In [ ]:
import numpy as np 
import time
from matplotlib  import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sys import exit

In [ ]:
def plot(bin_edges_x, bin_edges_y, x, y, field, fx, fy,a,b,u,v, plot_scatter=True, 
         plot_quiver=True, grid=True, plot_accquiver=True, cmap="viridis"):
    """
    A useful function for plotting parameters for the nbody assignment
    
    Most variables should be self explanatory.
    
    field should be for example the density field, potential field, forces... Whatever
    you think would be helpful to plot :)
    
    (fx, fy) are only for if you want to quiver plot and should be the force for each particle
    arranged in the same order as (x,y)
    """
    fig,ax = plt.subplots(figsize=(10,10), dpi=100)
    minor_ticks_x = bin_edges_x
    minor_ticks_y = bin_edges_y
    ax.set_xticks(minor_ticks_x, minor=False)
    ax.set_yticks(minor_ticks_y, minor=False)
    ax.set(xlim=(minor_ticks_x[0], minor_ticks_x[-1]), ylim=(minor_ticks_x[0], minor_ticks_x[-1]))
    if grid:
        ax.grid(which='both', alpha=0.75, color='w')        
        ax.set_xlabel("y")
        ax.set_ylabel("x")
    if plot_scatter:
        scat = ax.scatter(y, x, color='blue', s=50)
    cax = ax.imshow(
        field,
        # Extent goes (left, right, bottom, top)
        origin="lower",
        extent=(minor_ticks_x[0], minor_ticks_x[-1], minor_ticks_y[0], minor_ticks_y[-1]), 
        cmap=cmap
    )
    if plot_quiver:
        qax = ax.quiver(
            y, 
            x,
            fy, 
            fx,
            units="x",
            width=0.005*len(bin_edges_x),
            color="w"
        )
    if plot_accquiver:
        B, A = np.meshgrid(b,a)
        ax.quiver(B,A, v, u, units="x", width=0.005*len(bin_edges_x),color="w")
    plt.show()

In [ ]:
class particles:
    def __init__(self,x,y,z,vx,vy,vz,m=1.0,npart=1000,soft=0.37,dt=0.1,grid_min=-15,grid_max=15,n_grid=30):
        self.opts={}
        self.opts['soft']=soft
        self.opts['n']=npart
        self.opts['dt']=dt
        self.opts['grid_min']=grid_min
        self.opts['grid_max']=grid_max
        self.opts['n_grid']=n_grid
        self.x=x.copy()
        self.y=y.copy()
        self.z=z.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
        self.vx=vx.copy()
        self.vy=vy.copy()
        self.vz=vz.copy()
    def BCs(self,disappear=True,periodic=True):
        if periodic:
            for i in range(len(self.x)):
                if ( self.x[i] > self.opts['grid_max']): 
                    self.x[i] -= self.opts['n_grid']
                if ( self.x[i] < self.opts['grid_min']):
                    self.x[i] += self.opts['n_grid']
                if ( self.y[i] > self.opts['grid_max']): 
                    self.y[i] -= self.opts['n_grid']
                if ( self.y[i] < self.opts['grid_min']):
                    self.y[i] += self.opts['n_grid']
                if ( self.z[i] > self.opts['grid_max']): 
                    self.z[i] -= self.opts['n_grid']
                if ( self.z[i] < self.opts['grid_min']):
                    self.z[i] += self.opts['n_grid']
        if disappear:
            ind = []
            for i in range(len(self.x)):
                if (self.x[i]>self.opts['grid_max'] or self.x[i]<self.opts['grid_min'] or self.y[i]>self.opts['grid_max'] or self.y[i]<self.opts['grid_min'] or self.z[i]>self.opts['grid_max'] or self.z[i]<self.opts['grid_min']): 
                    ind.append(i)
            self.x = np.delete(self.x,ind,None)
            self.y = np.delete(self.y,ind,None)
            self.z = np.delete(self.z,ind,None)
            self.vx = np.delete(self.vx,ind,None)
            self.vy = np.delete(self.vy,ind,None)
            self.vz = np.delete(self.vz,ind,None)
            self.opts['n'] = len(self.x)       
            if len(self.x)==0:
                exit('All particles fell out of box!')
    def get_forces(self):
        self.fx=0*self.x
        self.fy=0*self.fx
        self.fz=0*self.fx
        pot_particles=0*self.fx
        pot=0
        points = (self.x,self.y,self.z)
        H, edges = np.histogramdd(points,bins=self.opts['n_grid'],range=((self.opts['grid_min'],self.opts['grid_max']),(self.opts['grid_min'], self.opts['grid_max']), (self.opts['grid_min'], self.opts['grid_max'])),)
        self.edges_x = edges[0]
        self.edges_y = edges[1]
        self.edges_z = edges[2]
        self.newedges_x = np.arange(self.opts['grid_min']+0.5,self.opts['grid_max']+0.5,1)
        self.newedges_y = np.arange(self.opts['grid_min']+0.5,self.opts['grid_max']+0.5,1)
        self.newedges_z = np.arange(self.opts['grid_min']+0.5,self.opts['grid_max']+0.5,1)
        edg = np.arange(2*self.opts['grid_min'],2*self.opts['grid_max']+1,1)
        particle_x_bins = np.digitize(self.x, self.edges_x, right=False) - 1
        particle_y_bins = np.digitize(self.y, self.edges_y, right=False) - 1
        particle_z_bins = np.digitize(self.z, self.edges_z, right=False) - 1
        self.rho = H*self.m
        rhopad=np.pad(self.rho,((self.opts['grid_max'],self.opts['grid_max']),(self.opts['grid_max'],self.opts['grid_max']),(self.opts['grid_max'],self.opts['grid_max'])), 'constant')
        rho_hat = np.fft.fftn(rhopad)
        X, Y, Z = np.meshgrid(edg[:-1],edg[:-1],edg[:-1]) 
        R = np.sqrt(X**2 + Y**2 + Z**2)
        R[R<self.opts['soft']]=self.opts['soft']
        GF = -1/(R + self.opts['soft'])
        GF_hat = np.fft.fftn(GF)
        phii = np.fft.ifftn(GF_hat*rho_hat)
        phipad = np.fft.ifftshift(phii).real         
        self.phi = phipad[self.opts['grid_max']:3*self.opts['grid_max'],self.opts['grid_max']:3*self.opts['grid_max'],self.opts['grid_max']:3*self.opts['grid_max']]
        self.accx = -np.gradient(self.phi, axis=0)         
        self.accy = -np.gradient(self.phi, axis=1)
        self.accz = -np.gradient(self.phi, axis=2)
        for q in range(self.opts['n']):
            self.fx[q] = self.m*self.accx[particle_x_bins[q], particle_y_bins[q], particle_z_bins[q]]      
            self.fy[q] = self.m*self.accy[particle_x_bins[q], particle_y_bins[q], particle_z_bins[q]]     
            self.fz[q] = self.m*self.accz[particle_x_bins[q], particle_y_bins[q], particle_z_bins[q]]     
            pot_particles[q] = self.m*self.phi[particle_x_bins[q], particle_y_bins[q], particle_z_bins[q]]
        pot = np.sum(pot_particles)
        return pot     
    def evolve(self):
        self.x += self.vx*self.opts['dt']
        self.y += self.vy*self.opts['dt']
        self.z += self.vz*self.opts['dt']
        self.BCs(disappear=False,periodic=True)
        pot=self.get_forces()
        self.vx += self.fx*self.opts['dt']
        self.vy += self.fy*self.opts['dt']
        self.vz += self.fz*self.opts['dt']
        kinetic = 0.5*np.sum(self.m*(self.vx**2+self.vy**2+self.vz**2))
        return pot,kinetic


n=10**5  
min = -15
max = 15
grid =30
x=(max - min)*np.random.random_sample((n,)) + min
y=(max - min)*np.random.random_sample((n,)) + min
z=(max - min)*np.random.random_sample((n,)) + min
vx=0*x
vy=vx.copy()
vz=vx.copy()

plt.ion()                                            
oversamp=5
#oversamp=11
part=particles(x,y,z,vx,vy,vz,m=1.,npart=n,dt=0.1/oversamp,grid_min=min,grid_max=max,n_grid=grid) 
fig, ax = plt.subplots()
ax.set_xlim(min, max)
ax.set_ylim(min, max)
plt.scatter(part.x,part.y,c='green',alpha=1.0,s=0.001)
plt.show()                                                  
pot=0
kin=0
for i in range(150):
    plt.clf()
    fig, ax = plt.subplots()
    ax.set_xlim(min, max)
    ax.set_ylim(min, max)
    plt.scatter(part.x,part.y,c='purple',alpha=1.0,s=0.001)                         
    plt.savefig("part3per{:03}.png".format(i))
    plt.pause(0.001)
    for j in range(oversamp):
        pot_old=pot
        kin_old=kin
        pot,kin=part.evolve()
    #print(pot+0.5*(kin+kin_old))
    #plot(part.edges_x,part.edges_y,part.x,part.y,part.rho3.sum(axis=2),part.fx,part.fy,
         #part.newedges_x,part.newedges_y,part.accx3.sum(axis=2),part.accy3.sum(axis=2),plot_quiver=False)   